<a href="https://colab.research.google.com/github/Nutsudapenpong/Confusion-matrix/blob/main/confusion_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json 
import pandas as pd
import numpy as np
from pathlib import Path
import PIL
from PIL import Image
import os, time, random
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh
from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import torch
from IPython.display import Image  # to display images
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(1) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.10.0+cu111 CPU


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## create csv files from txtx files
##  get label predict in Directory 
## convert .txt to .csv 
import glob
import csv

def create_file(name):
    ## get label predict in Directory 
    #PATH = f'/media/tohn/SSD_ex/YOLOv5/result_5m/prostate_prosmise12/scorethr_e400_prosmise12testset/{name}/labels/'
    #PATH = f'/content/drive/MyDrive/image_yolo/yolov5/runs/val/{name}/labels/'
    PATH = f'/content/drive/MyDrive/image_yolo/yolov5/test/labels/'
    path_txt = glob.glob(f"{PATH}*.txt")
    os.chdir(PATH)
    for p in path_txt:
        with open(p, 'r') as in_file:
            stripped = (line.strip() for line in in_file)
            lines = (line.split(" ") for line in stripped if line)
            name = p.rsplit('.',1)[0]
            name = name +'.csv'
            with open(name, 'w') as out_file:
                writer = csv.writer(out_file)
                writer.writerow(('class', 'x-center', 'y-center', 'w', 'h', 'conf'))
                writer.writerows(lines)   

In [6]:
#create csv file
name_file = ['exp26', 'exp27', 'exp28', 'exp29', 'with', 'without']
#exp26 = create_file(name_file[0])
#exp27 = create_file(name_file[1])
#exp28 = create_file(name_file[2])
#exp29 = create_file(name_file[3])
#with_ = create_file(name_file[4])
#without = create_file(name_file[5])

In [7]:
##### Concat CSV to 1 file
def concat_df(PATH, name_i) : ##path/to/lable/.csv/  ## name/scorethr/ 
    path_csv = glob.glob(f"{PATH}*.csv")
    df_  = list()
    for c in path_csv:
        print(c)
        df = pd.read_csv(c)
        print(df)
        image = c.split('/')[-1]
        print(image)
        image = image.rsplit('.',1)[0]+'.jpg'
        print(image)
        df['fileName'] = image
        df_.append(df)
    all_csv = pd.concat(df_ ,axis=0).reset_index(drop=True)
    all_csv.to_csv(f'{PATH}prosmise12-{name_i}.csv')
    ## Return/csv/table/all/score/thr
    return all_csv 

In [8]:
#already concatenate
#name_file = ['without']
#PATH = f'/content/drive/MyDrive/image_yolo/yolov5/test/labels/'
#concat_df(PATH, name_file[0])

In [9]:
#already concatenate
#name_file = ['exp27']
#PATH = f'/content/drive/MyDrive/image_yolo/yolov5/runs/val/exp27/labels/'
#concat_df(PATH, name_file[0])

In [10]:
#already concatenate
#name_file = ['exp29']
#PATH = f'/content/drive/MyDrive/image_yolo/yolov5/runs/val/exp29/labels/'
#concat_df(PATH, name_file[0])

In [11]:
#install rebox
!pip install rebox

In [12]:
from rebox import BBox
from rebox.formats import yolo, coco, pascal

# DataFrame0 = all_csv
def rebox(DataFrame0) :  ## DataFrame0 from concat_df
    #Opath = '/media/tohn/SSD_ex/ProstateCancer/PROMISE12_image_50_50/test/'
    Opath = '/content/drive/MyDrive/image_yolo/yolov5/test/images/'
    ###เก็บค่า shape imagesของ GT Bbox
    image_width_ = list()  #-- shape[1]
    image_height_ = list()  #-- shape[0]
    # image_height = 640 # pixels
    # image_width = 640 # pixels
    x_min_ = list()
    y_min_ = list()
    x_MAX_= list()
    y_MAX_ = list()

    for i in range(len(DataFrame0)) :
        x_Center = DataFrame0['x-center'][i] 
        y_Center = DataFrame0['y-center'][i]
        width = DataFrame0['w'][i] 
        height  = DataFrame0['h'][i]
        img_path_i = Opath+DataFrame0.fileName[i]
        image_width = cv2.imread(img_path_i).shape[1]
        image_height = cv2.imread(img_path_i).shape[0]
        yolo_bbox = BBox([x_Center, y_Center, width, height], yolo) # using built-in 'yolo_format'
        coco_bbox = yolo_bbox.as_format(coco, image_width, image_height) # to convert to built-in format 'coco'
        pascal_bbox = coco_bbox.as_format(pascal)
        bbox = pascal_bbox.value
        x_min = bbox[0]
        x_min_.append(x_min)
        y_min = bbox[1]
        y_min_.append(y_min)
        x_MAX = bbox[2]
        x_MAX_.append(x_MAX)
        y_MAX = bbox[3]
        y_MAX_.append(y_MAX)

    DataFrame0['pred_xmin'] = x_min_
    DataFrame0['pred_ymin'] = y_min_
    DataFrame0['pred_xmax'] = x_MAX_
    DataFrame0['pred_ymax'] = y_MAX_

    return DataFrame0 

In [13]:
# test
def reboxgt(DataFrame0) :  ## DataFrame0 from concat_df
    #Opath = '/media/tohn/SSD_ex/ProstateCancer/PROMISE12_image_50_50/test/'
    Opath = '/content/drive/MyDrive/image_yolo/yolov5/test/images/'
    ###เก็บค่า shape imagesของ GT Bbox
    image_width_ = list()  #-- shape[1]
    image_height_ = list()  #-- shape[0]
    # image_height = 640 # pixels
    # image_width = 640 # pixels
    x_min_ = list()
    y_min_ = list()
    x_MAX_= list()
    y_MAX_ = list()

    for i in range(len(DataFrame0)) :
        left = DataFrame0['x-center'][i] 
        top = DataFrame0['y-center'][i]
        w = DataFrame0['w'][i] 
        h  = DataFrame0['h'][i]
        coco_bbox = BBox([left,top,w,h], coco)
        pascal_bbox = coco_bbox.as_format(pascal)
        bbox = pascal_bbox.value
        x_min = bbox[0]
        x_min_.append(x_min)
        y_min = bbox[1]
        y_min_.append(y_min)
        x_MAX = bbox[2]
        x_MAX_.append(x_MAX)
        y_MAX = bbox[3]
        y_MAX_.append(y_MAX)

    DataFrame0['gt_xmin'] = x_min_
    DataFrame0['gt_ymin'] = y_min_
    DataFrame0['gt_xmax'] = x_MAX_
    DataFrame0['gt_ymax'] = y_MAX_

    return DataFrame0 

In [14]:
#รวม prediction box ใน 1 image 
def Get_1boxPre(table) :
    a = table.groupby(['fileName']).count()
    a = table.iloc[:,0]
    aa_ =[]
    for  i in  range(len(a)):
        aa = a[i]
        aa_.append(aa)
    name = sorted(list(set(table['fileName'])))

    fileName_, x1s_, y1s_, x2s_, y2s_, score_, =[], [], [], [], [], []
    for j in name:
        x1_, y1_, x2_, y2_, score = [], [], [], [], []
        fileName_.append(j)
        for i in range(len(table)):
            fileName = table['fileName'][i]
            if fileName == j:
                x1_.append(table['pred_xmin'][i])
                y1_.append(table['pred_ymin'][i])
                x2_.append(table['pred_xmax'][i])
                y2_.append(table['pred_ymax'][i])
                score.append(table['conf'][i])

        x1s_.append(x1_)
        y1s_.append(y1_)
        x2s_.append(x2_)
        y2s_.append(y2_)
        score_.append(score)

    result = {'fileName': fileName_, 'pred_xmin': x1s_,'pred_ymin': y1s_,'pred_xmax': x2s_,'pred_ymax': y2s_, 'pred_score': score_}
    result = pd.DataFrame(result, columns=['fileName', 'pred_xmin','pred_ymin','pred_xmax','pred_ymax', 'pred_score','pred_count'])

    count_ = list()
    for k in range(len(result)):
        count = len(result['pred_score'][k])
        count_.append(count)

    result['pred_count'] = count_

    return result 

In [15]:
#รวม actual box ใน 1 image 
def Get_1boxAct(test) :
    
    a = test.groupby(['fileName']).count()
    a = a.iloc[:,0]
    aa_ = []
    for  i in  range(len(a)):
        aa = a[i]
        aa_.append(aa)
    name = sorted(list(set(test['fileName'])))
    path = f'/content/drive/MyDrive/image_yolo/yolov5/test/images/{name}'
    #path = sorted(list(set(test['img_path'])))

    fileName_, x1s_, y1s_, x2s_, y2s_ = [], [], [], [], []
    for j in name:
        x1_, y1_, x2_, y2_ = [], [], [], []
        fileName_.append(j)
        for i in range(len(test)):
            fileName = test['fileName'][i] 
            if fileName == j:
                x1_.append(test['gt_xmin'][i])
                y1_.append(test['gt_ymin'][i])
                x2_.append(test['gt_xmax'][i])
                y2_.append(test['gt_ymax'][i])

        x1s_.append(x1_)
        y1s_.append(y1_)
        x2s_.append(x2_)
        y2s_.append(y2_)
        #Path_.append(test['img_path'][i])

    test_ = {'fileName': fileName_, 'Path':path, 'gt_xmin': x1s_,'gt_ymin': y1s_,'gt_xmax': x2s_,'gt_ymax': y2s_, 'count': aa_}
    test_ = pd.DataFrame(test_, columns=['fileName', 'Path', 'gt_xmin','gt_ymin','gt_xmax','gt_ymax', 'count'])

#     print(test_.shape)
#     test_.head()
    return test_

In [16]:
def map_table(test_, result_rebox) :## Paremeter -->>> test_ == test dataframe, result == predict dataframe 
    ## merge table 
    df_merge = pd.merge(test_, result_rebox, how="outer", on=["fileName", "fileName"])

    return df_merge

In [17]:
#tb = df_merge
def fill_(df_merge):
    columns_ = ['pred_xmin', 'pred_ymin', 'pred_xmax', 'pred_ymax', 'pred_score']
    for col in columns_: 
        for row in df_merge.loc[df_merge[col].isnull(), col].index:   ##for columns type is list 
            df_merge.at[row, col] = []

    for row in df_merge.loc[df_merge.pred_count.isnull(), 'pred_count'].index: ##for columns type is float
        df_merge.at[row, 'pred_count'] = 0

    return df_merge

In [18]:
def IoUfunction(res):
    pre_ ,  gt_ = [], []
    i = 0

    for j in range(len(res['pred_xmin'].tolist()[i])):
        xmin_pre = int(res['pred_xmin'].tolist()[i][j])
        ymin_pre = int(res['pred_ymin'].tolist()[i][j])
        xmax_pre = int(res['pred_xmax'].tolist()[i][j])
        ymax_pre = int(res['pred_ymax'].tolist()[i][j])
        pre = [xmin_pre, ymin_pre, xmax_pre, ymax_pre]
        pre_.append(pre)

    for j in range(len(res['gt_xmin'].tolist()[i])):
        xmin_gt= int(res['gt_xmin'].tolist()[i][j])
        ymin_gt = int(res['gt_ymin'].tolist()[i][j])
        xmax_gt= int(res['gt_xmax'].tolist()[i][j])
        ymax_gt = int(res['gt_ymax'].tolist()[i][j])
        gt = [xmin_gt, ymin_gt, xmax_gt, ymax_gt]
        gt_.append(gt)

    #ป้องกัน ไม่มีผล predict
    if pre_ == []:   ##if pre_ == np.NaN ## check NaN value()
        iou_pred = torch.tensor([0], dtype=torch.float)
        iou_gt = torch.tensor([0], dtype=torch.float)
    else:
        box1 = torch.tensor(pre_, dtype=torch.float)
        box2 = torch.tensor(gt_, dtype=torch.float)
        iou_pred = bops.box_iou(box1, box2)
        iou_gt = bops.box_iou(box2, box1)

    return iou_pred, iou_gt

In [19]:
import torch
import torchvision.ops.boxes as bops

def EvaluationTeble(result, threshold_iou):
    IoU_pred, IoUavg_img_ = [], []
    fileName = result['fileName'].tolist()

    TP, FP, FN = [], [], []
    for i in fileName:   
        #IoU Function
        res = result[result['fileName'] == i ]
        Gt_count = res['count'].tolist()[0]
        pre_count = res['pred_count'].tolist()[0]
        iou_pred, iou_gt = IoUfunction(res)
        
        if pre_count == 0 :
            TP.append(0)
            FP.append(0)
            FN.append(Gt_count) 
            IoU_pred.append(torch.tensor([0], dtype=torch.float))
        else:
            '''Box Predict checking '''
#             iou_PdGt_ = []
#             IoUavg_im = []
            iou_tp_ =  []
            iou_fp_ =  []
            '''1 . TP = number of detections with IoU>=0.5 '''
            for i in range(len(iou_pred)):  ### 
                iou_tp = max(iou_pred[i])
                if iou_tp == threshold_iou:
                    iou_tp = 1
                    iou_tp_.append(iou_tp)
                elif iou_tp > threshold_iou:
                    iou_tp = 1
                    iou_tp_.append(iou_tp)

            ''' 2 . FP = number of detections with IoU<0.5 or detected more than once '''
            for i in range(len(iou_pred)):  ### 
                iou_fp = max(iou_pred[i])
                if iou_fp < threshold_iou:
                    iou_fp = 0
                    iou_fp_.append(iou_fp)

            #TP,FP
            tp, fp = 0, 0
            for i in list(set(iou_tp_)):
                if len(iou_tp_) != 0:
                    i==1
                    tp = iou_tp_.count(i)
                else:
                    tp=0
                    
            for j in list(set(iou_fp_)):
                if len(iou_fp_) != 0:
                    j==0
                    fp = iou_fp_.count(j)
                else:
                    fp=0
                    
            if tp > Gt_count:
                tp_gt = tp-Gt_count
                tp = Gt_count
                fp = fp+tp_gt

            ''' 3 . FN = number of detections with IoU<0.5 '''
            fn = Gt_count-tp            
            
            TP.append(tp)
            FP.append(fp)
            FN.append(fn) 
            IoU_pred.append(iou_pred)

    result['IoU_img'] = IoU_pred
    result['TP'] = TP
    result['FP'] = FP
    result['FN'] = FN

    return result

Start here

In [ ]:
import glob
import csv

##Import Test set dataframe
test = pd.read_csv('/content/drive/MyDrive/image_yolo/yolov5/runs/val/promise12/without.csv')
DataFrame0 = pd.read_csv('/content/drive/MyDrive/image_yolo/yolov5/runs/val/promise12/exp29.csv')
threshold_score = [0.60]
#name_thr = ['PCI0','PCI01','PCI02','PCI03','PCI04','PCI05','PCI06','PCI07','PCI08','PCI09']
name_thr = ['exp29']
precision_, recall_, sum_TP_, sum_FP_, sum_FN_, sum_count_, sum_pred_count_ ,name_thr_, threshold_score_ = [],[],[],[],[],[],[],[],[]
sum_count, sum_pred_count = [],[]
threshold_iou = 0.50
### Check file 
for i, n in zip(range(len(threshold_score)), range(len(name_thr))):
    name_i = name_thr[n]
    threshold_score_i = threshold_score[i]
    PATH = f'/content/drive/MyDrive/image_yolo/yolov5/runs/val/{name_i}/labels/'
    path_txt = glob.glob(f"{PATH}*.txt")
    if len(path_txt) != 0:
        print("____")
        table = rebox(DataFrame0)
        result_rebox = Get_1boxPre(table)
        table_ = reboxgt(test)
        test_ = Get_1boxAct(table_)
        df_merge = map_table(test_, result_rebox)
        result = fill_(df_merge)
        Eva_table = EvaluationTeble(result, threshold_iou)
        ### Calculate perfermance matrix 
        sum_TP = sum(list(Eva_table.TP))
        sum_FP = sum(list(Eva_table.FP))
        sum_FN = sum(list(Eva_table.FN))
        sum_count = sum(list(Eva_table['count']))
        sum_pred_count = sum(list(Eva_table.pred_count))
        recall = sum_TP/(sum_TP+sum_FN)
        precision = sum_TP/(sum_TP+sum_FP)
        ### Append perfermance to list 
        precision_.append(precision)
        recall_.append(recall)
        sum_TP_.append(sum_TP)
        sum_FP_.append(sum_FP)
        sum_FN_.append(sum_FN)
        sum_count_.append(sum_count)
        sum_pred_count_.append(sum_pred_count)
    elif len(path_txt) == 0:
        sum_TP = 0
        sum_FP = 0
        sum_FN = 0
        recall = 0
        precision = 0
        sum_pred_count = 0
        sum_count = sum(list(test_['count']))
        ### Append perfermance to list 
        precision_.append(precision)
        recall_.append(recall)
        sum_TP_.append(sum_TP)
        sum_FP_.append(sum_FP)
        sum_FN_.append(sum_FN)
        sum_count_.append(sum_count)
        sum_pred_count_.append(sum_pred_count)
        
    name_thr_.append(name_i)
    threshold_score_ .append(threshold_score_i)

____


KeyboardInterrupt: ignored

In [ ]:
print(result_rebox)

In [ ]:
print(test_)

In [ ]:
df_merge.head()

In [ ]:
print(len(precision_))
print(len(recall_))
print(len(sum_TP_))
print(len(sum_FP_))
print(len(sum_FN_))
print(len(sum_count_))
print(len(sum_pred_count_))
print(len(threshold_score_))

In [ ]:
precision_

In [ ]:
recall_

In [ ]:
i = np.arange(len(threshold_score_))
roc0 = pd.DataFrame({'Threshold' : pd.Series(threshold_score_, index=i),'Precision' : pd.Series(precision_, index = i),'Recall' : pd.Series(recall_, index = i),
                     'TP' : pd.Series(sum_TP_, index = i), 'FP' : pd.Series(sum_FP_, index = i), 'FN' : pd.Series(sum_FN_, index = i), 
                     'GT count' : pd.Series(sum_count_, index = i), 'Pre count' : pd.Series(sum_pred_count_, index = i)})
roc0

In [ ]:
roc0 = roc0.sort_values(by='Threshold', ascending=True)
#fix_value = [{'Threshold':0.000,'Precision':0.000,'Recall':1.000}]
#roc0 = roc0.append(fix_value,ignore_index=True,sort=False)
#roc0 = roc0.apply(np.roll, shift=1) #move he last row to the first
roc0.loc[roc0.index[-1], 'Precision'] = 1.000
roc0.loc[roc0.index[-1], 'Recall'] = 0.000
roc0.loc[roc0.index[-1], 'TP'] = 0
roc0.loc[roc0.index[-1], 'FP'] = 0
roc0.loc[roc0.index[-1], 'FN'] = 0
roc0.loc[roc0.index[-1], 'GT count'] = 19
roc0.loc[roc0.index[-1], 'Pre count'] = 0
roc0

In [ ]:
Prec = roc0['Precision'].tolist()
rec = roc0['Recall'].tolist()
rec = np.array(rec)
Prec = np.array(Prec)

from sklearn import metrics
#auc = metrics.auc(Prec, rec)
#auc = metrics.auc(rec, Prec)
print("Area Under Curve: %0.4f" % auc)

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(8, 5))
# plotting points as a scatter plot
plt.fill_between(rec, Prec, color='#95d0fc', alpha=0.4)
plt.plot(rec, Prec, label= "Precision-Recall Curve: AUC=%0.4f"%auc, color= "blue",
                marker= ".",markerfacecolor='blue', markersize=10, alpha=0.4)

# x-axis label
plt.xlabel('Recall')
# frequency label
plt.ylabel('Precision')
# plot title
plt.title('Precision-Recall curve')
# showing legend
plt.legend()

# function to show the plot
plt.show()


In [ ]:
DataFrame0  = pd.read_csv(f'/media/tohn/SSD/YOLOv5/result_5s/prostate/scorethr_e400/PCI03/labels/prostate_custom_PCI03.csv')
table = rebox(DataFrame0)
result_rebox = Get_1boxPre(table)
test_ = Get_1boxAct(test)
df_merge = map_table(test_, result_rebox)
result = fill_(df_merge)
Eva_table = EvaluationTeble(result, threshold_iou)
### Calculate perfermance matrix 
sum_TP = sum(list(Eva_table.TP))
sum_FP = sum(list(Eva_table.FP))
sum_FN = sum(list(Eva_table.FN))
sum_count = sum(list(Eva_table['count']))
sum_pred_count = sum(list(Eva_table.pred_count))
recall = sum_TP/(sum_TP+sum_FN)
precision = sum_TP/(sum_TP+sum_FP)

In [ ]:
IoU_pred = Eva_table['IoU_img'].tolist()
aa = []
for m in IoU_pred: 
    if m.shape[0] >1:
        a_ =[]
        for i in range(m.shape[1]):
            a = max(m[0][i],m[1][i])
            a_.append(float(a))
        a = np.mean(a_)
        aa.append(a)
    else:
        a = m.mean()
        aa.append(float(a))

print('\n', 'IoU :', aa)
print('\n', 'IoU Avg :', np.mean(aa))

In [ ]:
print(f'True Positive ** SUM = {sum_TP}')
print(f'{list(Eva_table.TP)}')
print('-' * 50)
print(f'False  Positive ** SUM = {sum_FP}')
print(f'{list(Eva_table.FP)}')
print('-' * 50)
print(f'False Negative  ** SUM = {sum_FN}')
print(f'{list(Eva_table.FN)}\n')
print('-' * 50)
print(f'Precision ** : {precision}\n')
print(f'Recall ** : {recall}\n')
print('-' * 50)

print(f' Result Dataframe {Eva_table.shape}')
Eva_table

In [ ]:
print(f'Number** Bbox Prediction TP : {int(sum(Eva_table.TP))}\n')
sum_count0 = sum(list(Eva_table['count']))
print(f'Number** Bbox ground Truth : {int(sum_count0)}')

In [ ]:
neg = np.array([[0, sum_FP]]) ## TN, FP 
pos = np.array([[sum_FN, sum_TP]]) ## FN, TP
cf_matrix = np.concatenate((neg, pos), axis=0)
cf_matrix
print(cf_matrix)
#labels = [‘True Neg’,’False Pos’,’False Neg’,’True Pos’]

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
fig, ax = plt.subplots(figsize=(8, 5))

ax = sns.heatmap(cf_matrix, annot=True, fmt="d", cmap="YlGnBu") #Blues,Oranges,Reds
ax.set_title('Confusion matrix',fontsize=20)
ax.set_ylabel('True label',fontsize=18)
ax.set_xlabel('Predicted label',fontsize=18)

In [ ]:
group_names = ['True Neg','False Pos','False Neg','True Pos' ]
group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
#xyplotlabels = ['True Label' ,'Predicted Label']
# group_percentages = ['{0:.2%}'.format(value) for value in
#                      cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}' for v1, v2 in
          zip(group_names,group_counts)]
labels = np.asarray(labels).reshape(2,2)
plt.figure(figsize=(8,5))
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
plt.ylabel('True label',fontsize=18)
plt.xlabel('Predicted label', fontsize=18)
plt.title('Confusion Matrix', fontsize=20)

In [ ]:
# Plot img Prediction
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams["axes.grid"] = False

res = Eva_table
w = 20
h = 20
fig = plt.figure(figsize=(30, 10))
columns = 5
rows = 2

fileName = Eva_table['fileName'].tolist()
for m in range(1, columns*rows +1):
    res = Eva_table[Eva_table['fileName'] ==fileName[m-1]]

    i = 0
    #load image origin save to test.jpg
    import cv2
    from IPython.display import Image
    img_path = res['Path'].tolist()[0]
    img_c = cv2.imread(img_path)

    for j in range(len(res['gt_xmin'].tolist()[i])):
        xmin_pre = int(res['gt_xmin'].tolist()[i][j])
        ymin_pre = int(res['gt_ymin'].tolist()[i][j])
        xmax_pre = int(res['gt_xmax'].tolist()[i][j])
        ymax_pre = int(res['gt_ymax'].tolist()[i][j])
        if j == 0:
            image_pre = cv2.rectangle(img_c, (xmin_pre ,ymin_pre), (xmax_pre, ymax_pre), (0,255,0), 6)
        else:
            image_pre = cv2.rectangle(image_pre, (xmin_pre ,ymin_pre), (xmax_pre, ymax_pre), (0,255,0), 6)

    for j in range(len(res['pred_xmin'].tolist()[i])):
        xmin_pre = int(res['pred_xmin'].tolist()[i][j])
        ymin_pre = int(res['pred_ymin'].tolist()[i][j])
        xmax_pre = int(res['pred_xmax'].tolist()[i][j])
        ymax_pre = int(res['pred_ymax'].tolist()[i][j])
        if j == 0:
            image_pre = cv2.rectangle(image_pre, (xmin_pre ,ymin_pre), (xmax_pre, ymax_pre), (255, 0, 0), 4)
        else:
            image_pre = cv2.rectangle(image_pre, (xmin_pre ,ymin_pre), (xmax_pre, ymax_pre), (255, 0, 0), 4)

    fig.add_subplot(rows, columns, m)
    plt.imshow(image_pre)
    plt.axis('off')
        
plt.show()